In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from math import sqrt
from sklearn import model_selection
from scipy.stats import spearmanr

In [2]:
# Read Vectors Used as Input Features
inp_vectors = pd.read_csv("rand_forest.csv")
# Read Vectors Used as Target Features
target_data = pd.read_csv('properties.csv')
# Prepare to Read Train and Testing Masks as Lists
train_masks = []
test_masks = []

In [3]:
# Seperate the Names of Cofs from Dataframe
# Use for future alignment of Target and Input DB order
db_cofs = inp_vectors[['cof']]
# Create Single Dimentional Array
db_cofs = np.squeeze(db_cofs.values)

In [4]:
# Seperate the Training Features from Names
db_traindata = inp_vectors[['ASA_m^2/g','Density','LS','B','O','C','H',
                        'Si','N','S','Ni','Zn','Cu','Co','F','P','Cl','V','Br']]
# Convert to Arrays
db_traindata = db_traindata.values

In [5]:
# Read in The Train and Test Masks from Seperate pkl Files
# Stored in the './splits' directory
for i in range(10):
    # Read in Object from pkl
    obj = pd.read_pickle(r'splits/split_run_{}.pkl'.format(i))
    # Seperate Out Masks as Keys
    train_masks.append(obj['masks']['train'])
    test_masks.append(obj['masks']['test'])

First, remove the values in target that don't have valid graphs

In [6]:
# Convert Names from Targets and Inputs into lists
# Then use set Overlap in order to determine values we don't need
# Some are not present in splits because of Graph Bugs
list1 = np.squeeze(target_data[['name']].values)
list2 = db_cofs
# valid_list is the list of overlapping (and thus valid) cofs
valid_list = list(set(list1).intersection(list2))

In [7]:
# Extract the targets from the Matrix Factorization paper
target_data = target_data[['name','h2o_henry', 'h2s_henry', 'xe_henry', 'kr_henry', 'co2_0.001bar', 'o2_5bar', 'o2_140bar', 'co2_30bar', 'n2_0.001bar', 'n2_30bar', 'h2_77K_5bar', 'h2_77K_100bar', 
            'h2_298K_5bar', 'h2_298K_100bar', 'ch4_65bar', 'ch4_5.8bar']]
# Convert to Arrays
target_data = target_data.values

In [9]:
# Define a list called removal, this will hold indeces we need to remove
removal = []
# Sift through all of the target data
for i,cof in enumerate(target_data):
    # If the name is not valid, then add the index to the removal list
    if cof[0] not in valid_list:
        removal.append(i)

In [10]:
# Now use the numpy delete function in order to remove values at those indeces
# Now all of the target data is what we would like to use!
target_data = np.delete(target_data, removal, 0)

Then, use the targets to produce an ordered set of input vectors

In [11]:
# Make an empty list called sorted_cofs, which will hold indeces that we would like
# to reorganize our list of training data to. 
sorted_cofs = []
for name in list2:
    # Find the index that we need to shift the current value to, based on the target organization
    sorted_cofs.append(np.where(target_data[:,0] == name))
# Make sure the dimentions are right
sorted_cofs = np.squeeze(sorted_cofs)

In [12]:
# Now that we are organized, remove the names from the targets
target_data = target_data[:,1:]

In [13]:
# Shift the training data using the new indeces
db_traindata = db_traindata[sorted_cofs]

Seperate Training and Testing Sets

In [14]:
# Prepare a matrix of indeces, each one of the slots will hold an arrayu
# of indeces in which the training set is contained for that target
train_ones = np.empty((10,16), dtype=object)
test_ones = np.empty((10,16), dtype=object)

In [15]:
# Now at each index, make a test and train set of indeces
for i in range(10):
    for j in range(16):
        # Find all of the ones in the train mask, these indeces will be the training set
        train_ones[i][j] = np.where(train_masks[i][:,j] == 1)[0]
        # Find all of the ones in the test mask, these indeces will be the testing set
        test_ones[i][j] = np.where(test_masks[i][:,j] == 1)[0]

In [16]:
# Now prepare empty arrays that will hold the training set, and testing set, for each target and each split
train_inputs = np.empty((10,16), dtype=object)
test_inputs = np.empty((10,16), dtype=object)
train_outputs = np.empty((10,16), dtype=object)
test_outputs = np.empty((10,16), dtype=object)

In [17]:
# Use the mask indeces we found before in order to generate training and testing sets
for i in range(10):
    for j in range(16):
        # Regardless of target, we will get all of the input data
        train_inputs[i][j] = np.array([db_traindata[i] for i in train_ones[i][j]],dtype=np.float64)
        test_inputs[i][j] = np.array([db_traindata[i] for i in test_ones[i][j]],dtype=np.float64)
        # On the output data, we will need an additional index to specify which target we are looking at
        train_outputs[i][j] = np.array([target_data[i][j] for i in train_ones[i][j]],dtype=np.float64)
        test_outputs[i][j] = np.array([target_data[i][j] for i in test_ones[i][j]],dtype=np.float64)

Standardize Targets

In [18]:
# Each index, since correlated to a target, will have a mean, and absolute deviation
means = np.empty((10,16), dtype=np.float64)
stdevs = np.empty((10,16), dtype=np.float64)

In [19]:
# Calculate the mean and stdevs, then push into the corresponding slot
for i in range(10):
    for j in range(16):
        means[i][j] = train_outputs[i][j].mean()
        stdevs[i][j] = np.std(train_outputs[i][j])

In [20]:
# Create arrays to store the standardized (z-score) version of the training and testing outputs
train_outputs_z = np.empty((10,16), dtype=object)
test_outputs_z = np.empty((10,16), dtype=object)

In [21]:
# Basic Standardization Procedure, dimentions of outputs do not change
for i in range(10):
    for j in range(16):
        train_outputs_z[i][j] = (train_outputs[i][j] - means[i][j]) / stdevs[i][j]
        test_outputs_z[i][j] = (test_outputs[i][j] - means[i][j]) / stdevs[i][j]

Implement Random Forest Model

In [22]:
# Create a variable of the target names for ease of printing
target_name = ['h2o_henry', 'h2s_henry', 'xe_henry', 'kr_henry', 'co2_0.001bar', 'o2_5bar', 'o2_140bar', 'co2_30bar', 'n2_0.001bar', 'n2_30bar', 'h2_77K_5bar', 'h2_77K_100bar', 
            'h2_298K_5bar', 'h2_298K_100bar', 'ch4_65bar', 'ch4_5.8bar']

In [33]:
# Get ready to store all of the performance metric for each test that is run
MAEs = np.empty((10,16), dtype=np.float64)
SPRs = np.empty((10,16), dtype=np.float64)
MSEs = np.empty((10,16), dtype=np.float64)
RMSEs = np.empty((10,16), dtype=np.float64)

In [30]:
# Now run all 10 x 16 tests in sequence, using the test and train splits we made
for i in range(10):
    # Print the epoch to get some idea of what is going on
    print("Epoch: ", i)
    for j in range(16):
        # Instanciate the model, baseline model taken from another project (for now)
        Random_Forest = ExtraTreesRegressor(n_estimators = 200, random_state = 0, criterion = "absolute_error", bootstrap = True, warm_start = True)
        # Fit the model using the inputs and zscored outputs
        Random_Forest.fit(train_inputs[i][j], train_outputs_z[i][j])
        # Make predictions using test inputs
        test_preds = Random_Forest.predict(test_inputs[i][j])
        # Find the metrics
        MAEs[i][j] = metrics.mean_absolute_error(test_outputs_z[i][j], test_preds)
        SPRs[i][j] = spearmanr(test_outputs_z[i][j], test_preds)[0]
        MSEs[i][j] = metrics.mean_squared_error(test_outputs_z[i][j], test_preds)
        RMSEs[i][j] = np.sqrt(metrics.mean_squared_error(test_preds, test_outputs_z[i][j]))

Epoch:  0
Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9


In [31]:
# Take averages over the 10 runs for each target
MAEs = MAEs.mean(axis=0)
SPRs = SPRs.mean(axis=0)
MSEs = MSEs.mean(axis=0)
RMSEs = RMSEs.mean(axis=0)

In [32]:
# Tell the user what is going on
for i in range(16):
    print("Printing Details for Target: ", target_name[i])
    print("---------------------------")
    print("Average MAE: ", MAEs[i])
    print("Average MSE: ", MSEs[i])
    print("Average SPR: ", SPRs[i])
    print("Average RMSE: ", RMSEs[i])
    print("________________________________________________\n\n")
    
print("Overall Averages")
print("****************************************")
print("MAE: ", MAEs.mean())
print("MSE: ", MSEs.mean())
print("SPR: ", SPRs.mean())
print("RMSE: ", RMSEs.mean())
print("***************************************:)")

Printing Details for Target:  h2o_henry
---------------------------
Average MAE:  0.3166809137995439
Average MSE:  2.264971186212118
Average SPR:  0.016785990745714713
Average RMSE:  1.2856965641940135
________________________________________________


Printing Details for Target:  h2s_henry
---------------------------
Average MAE:  0.580170370886459
Average MSE:  1.6392833640760955
Average SPR:  -0.08200417496841512
Average RMSE:  1.2532152305004058
________________________________________________


Printing Details for Target:  xe_henry
---------------------------
Average MAE:  0.3685351077190487
Average MSE:  1.1695057680275134
Average SPR:  -0.016138672616099435
Average RMSE:  0.9720931988651372
________________________________________________


Printing Details for Target:  kr_henry
---------------------------
Average MAE:  0.6318335388739145
Average MSE:  1.4609884563782087
Average SPR:  -0.02828827714716467
Average RMSE:  1.1887101911178826
______________________________________